[current dir](.)

In [336]:
# with open('inputs/Day13-example.txt') as file:
# with open('inputs/Day13-example2.txt') as file:
with open('inputs/Day13.txt') as file:
    input = [list(line) for line in file.read().splitlines()]

In [413]:
from copy import copy

normalDir = {
    'left':     {'>': (0, -1), '<': (0, 1),  'v': (1, 0),  '^': (-1, 0)},
    'straight': {'>': (1, 0),  '<': (-1, 0), 'v': (0, 1), '^': (0, -1)},
    'right':    {'>': (0, 1),  '<': (0, -1), 'v': (-1, 0), '^': (1, 0)}
}
symbConv = {
    'left':     {'>': '^',  '^': '<', '<': 'v', 'v': '>'},
    'straight': {'>': '>',  '^': '^', '<': '<', 'v': 'v'},
    'right':    {'>': 'v',  '^': '>', '<': '^', 'v': '<'},
    '\\': {'>': 'v',  '^': '<', '<': '^', 'v': '>'},
    '/': {'>': '^',  '^': '>', '<': 'v', 'v': '<'},
    '|': {'^': '^', 'v': 'v'},
    '-': {'>': '>', '<': '<'}
}
rotations = ['left', 'straight', 'right']
symL, symC, symI = ['|', '-'], ['\\', '/'], ['+']
symLI, symLC, symCI, symLCI = symL + symI, symL + symC, symC + symI, symLC + symI
connVer = symCI + ['|']
connHor = symCI + ['-']

def printboard(b):
    print(*[''.join(line) for line in b], sep="\n")
    
def extract():
    carts = []
    board = [copy(r) for r in input]
    board.append([" " for _ in range(len(board[0]))])
    for y, row in enumerate(board):
        board[y].append(" ")
        for x, direction in enumerate(row):
            if direction in ['>', '<', 'v', '^']:
                carts.append((direction, x, y, 0))
                if (board[y-1][x] in connVer and board[y+1][x] in connVer 
                        and board[y][x-1] in connHor and board[y][x+1] in connHor):
                    board[y][x] = '+'
                elif ((board[y-1][x] in connVer and board[y][x-1] in connHor)
                         or (board[y+1][x] in connVer and board[y][x+1] in connHor)):
                    board[y][x] = '/'
                elif ((board[y-1][x] in connVer and board[y][x+1] in connHor)
                         or (board[y+1][x] in connVer and board[y][x-1] in connHor)):
                    board[y][x] = '\\'
                elif board[y-1][x] in connVer and board[y+1][x] in connVer:
                    board[y][x] = '|'
                elif board[y][x-1] in connHor and board[y][x+1] in connHor:
                    board[y][x] = '-'
    print(len(carts), "carts", "\n")
    return carts, board
    
def printwithcarts():
    withcarts = [copy(r) for r in board]
    for idx, (symbol, x, y, turns) in enumerate(carts):
        withcarts[y][x] = symbol
    printboard(withcarts)

Part 1

In [411]:
def part1():
    carts, board = extract()
    for tick in range(1000):            
        for idx, (direction, x, y, turns) in enumerate(carts):
            if any(ax == x and ay == y for idx2, (_, ax, ay, _) in enumerate(carts) if idx != idx2):
                return tick, x, y
            olddir = direction
            if board[y][x] == '+':
                rotation = rotations[turns]
                direction = symbConv[rotation][direction]
                turns = (turns + 1) % 3
            else:
                olddir = direction = symbConv[board[y][x]][direction]
                olddir = direction
                rotation = 'straight'
            (x, y) = [x + y for x, y in zip((x, y), normalDir[rotation][olddir])]
            carts[idx] = (direction, x, y, turns)
            
part1()

17 carts 



(164, 102, 114)

Part 2

This took me way too long to solve.
The author of the challenge did not mention that in such a situation ->>-, 
a crash does occur. This does not seem intuitive to me. And it computed the crash
by looking at the carts in increasing order of x, y.
So, in order to do efficient checks, I checked for collisions after each tick. This
should be fine. But because of this technicality I needed to check for collisions
within the inner cart loop, and only after ordering the cart list alphabetically.

This kind of omitted technicality should not occur in a good problem.

In [412]:
def part2():
    carts, board = extract()
    for tick in range(100000):            
        removed = []
        carts.sort(key=lambda x: x[1])
        carts.sort(key=lambda x: x[2])
        
        for idx, (direction, x, y, turns) in enumerate(carts):
            for idx2, (_, ax, ay, _) in enumerate(carts):
                if idx != idx2 and ax == x and ay == y:
                    removed.append(idx)
                    removed.append(idx2)
            olddir = direction
            if board[y][x] == '+':
                rotation = rotations[turns]
                direction = symbConv[rotation][direction]
                turns = (turns + 1) % 3
            else:
                olddir = direction = symbConv[board[y][x]][direction]
                olddir = direction
                rotation = 'straight'
            (x, y) = [x + y for x, y in zip((x, y), normalDir[rotation][olddir])]
            carts[idx] = (direction, x, y, turns)
        
        carts = [cart for index, cart in enumerate(carts) if index not in removed]
        if len(carts) == 1:
            return x, y
            
part2()

17 carts 



(146, 87)